In [59]:
import folium
import pandas as pd
from string import Template

In [60]:
df = pd.read_csv("munro_bagger - Sheet1 (2).csv").fillna("").astype(str)

In [61]:
for col in ["Metres", "Scott", "Matthew"]:
    df[col] = df[col].apply(lambda x: x.split(".")[0])

In [62]:
row_template = Template(
                        """<tr>
                            <td><button onclick="window.map.setView([$lat, $long], 13)">$name</button></td>
                            <td>$height</td>
                            <td>$images</td>
                          </tr>
                          """)

In [63]:
rows_html = ""

for _, row in df.iterrows():

    row_images = ""
    if row["Matthew"] != "":
        row_images += '<img src="images/matthew.png" class="uniform-img">'

    if row["Scott"] != "":
        row_images += '<img src="images/scott.png" class="uniform-img">'

    row = row_template.substitute({'name': row["Name"],
                            'lat': row["Latitude"],
                             'long': row["Longitude"],
                             'height': row["Metres"]+"m   ",
                             'images': row_images
                            })


    rows_html += row

In [64]:
scotland_center = [56.4907, -4.2026]
m = folium.Map(location=scotland_center, zoom_start=6)

map_init_js = """
<style>
#custom-controls {
  width: 520px;
  max-width: 90vw; /* Never overflow viewport */
  height: auto;
}

@media (max-width: 768px) {
  #custom-controls {
    width: 90vw;
    left: 5vw;
    top: 10px;
    bottom: auto;
    max-height: 80vh;
  }

  table {
    width: 100%;
    font-size: 0.9em;
  }

  td img, .uniform-img {
    width: 60px;
    height: 60px;
  }
}

#custom-controls table {
    width: 100%;
    table-layout: auto; /* or fixed if you want even column widths */
    border-collapse: collapse;
    word-break: break-word;
}

#custom-controls th, 
#custom-controls td {
    padding: 8px;
    text-align: left;
    min-width: 50px;
}

@media (max-width: 600px) {
    #custom-controls {
        max-width: 90vw;
    }
}

</style>

<script>
document.addEventListener("DOMContentLoaded", function() {
    for (var key in window) {
        if (key.startsWith("map_")) {
            window.map = window[key];
            break;
        }
    }
});
</script>
"""

# HTML control panel
panel_html_template = Template("""
<div id="custom-controls" style="
    position: absolute;
    top: 10px;
    left: 10px;
    bottom: 10px;
    z-index: 9999;
    background: white;
    padding: 10px;
    border-radius: 8px;
    box-shadow: 0 0 5px rgba(0,0,0,0.3);
    font-family: sans-serif;
    max-height: calc(100vh - 20px);
    overflow-y: auto;
    width: 100%;
    max-width: 27vw;
    min-width: 200px;
    box-sizing: border-box;
">
  <style>
    .uniform-img {
      width: 80px;
      height: 80px;
      object-fit: contain;
      background-color: transparent;
    }

    table {
      border-collapse: collapse;
      width: 80%;  /* 20% narrower */
      margin: 20px auto 0;
      font-family: Arial, sans-serif;
      text-align: left;
    }

    th, td {
      border: 1px solid #ccc;
      padding: 10px 15px;
      vertical-align: middle;
    }

    thead {
      background-color: #f2f2f2;
      position: sticky;
      top: 0;
      z-index: 1;
    }

    button {
      background-color: #007bff;
      color: white;
      border: none;
      padding: 7px 12px;
      cursor: pointer;
      font-size: 1em;
      border-radius: 4px;
    }

    button:hover {
      background-color: #0056b3;
    }

    td img {
      width: 80px;
      height: 80px;
      object-fit: cover;
      margin-right: 5px;
      vertical-align: middle;
      border-radius: 4px;
    }

    #searchInput {
      width: 100%;
      padding: 8px 10px;
      margin-bottom: 10px;
      font-size: 1em;
      border: 1px solid #ccc;
      border-radius: 4px;
    }
  </style>

  <!-- 🔍 Search Input -->
  <input type="text" id="searchInput" placeholder="Search munros...">

  <table id="mountainTable">
    <thead>
      <tr>
        <th>Name</th>
        <th>Height</th>
        <th>Bags</th>
      </tr>
    </thead>
    <tbody>
      $rows
    </tbody>
  </table>

  <!-- 🔍 Search Functionality -->
  <script>
    const searchInput = document.getElementById('searchInput');
    const table = document.getElementById('mountainTable');
    const rows = table.querySelectorAll('tbody tr');

    searchInput.addEventListener('input', function () {
      const filter = searchInput.value.toLowerCase();
      rows.forEach(row => {
        const nameCell = row.querySelector('td').innerText.toLowerCase();
        row.style.display = nameCell.includes(filter) ? '' : 'none';
      });
    });
  </script>
</div>
"""
)

panel_html = panel_html_template.substitute({"rows": rows_html})



# Inject both
m.get_root().html.add_child(folium.Element(map_init_js))
m.get_root().html.add_child(folium.Element(panel_html))

In [65]:
matthew_bag = folium.CustomIcon(icon_image="images/matthew.png",
                            icon_size=(16, 16),
                            icon_anchor=(15, 16)
                            )

scott_bag = folium.CustomIcon(icon_image="images/scott.png",
                            icon_size=(16, 16),
                            icon_anchor=(1, 16)
                            )

peak = folium.CustomIcon(icon_image="images/peak.png",
                            icon_size=(16, 16),
                            icon_anchor=(8, 8)
                            )

In [66]:
bag_icons = {"Matthew": matthew_bag,
             "Scott": scott_bag,
            }

In [67]:
def add_peak(row):

    name = row["Name"]
    height = row["Metres"]
    coords = [row["Latitude"], row["Longitude"]]

    label = "<br>".join([name, f"height: {height}m"])

    icon = peak

    folium.Marker(
                location = coords,
                tooltip = label,
                icon = icon
                ).add_to(m)

    return

In [68]:
def add_bag(row):

    year = row[person]
    icon = bag_icons[person]
    coords = [row["Latitude"], row["Longitude"]]
    
    folium.Marker(
    location = coords,
    tooltip = year,
    icon = icon
    ).add_to(m)

    return

In [69]:
for _, row in df.iterrows():
    add_peak(row)

In [70]:
for person in ["Matthew", "Scott"]:
    
    for _, row in df[df[person] != ""].iterrows():
        add_bag(row)

In [71]:
# Save it
m.save("index.html")

In [ ]:
scotland_center = [56.4907, -4.2026]
m = folium.Map(location=scotland_center, zoom_start=6)

# JavaScript to extract the internal map object and assign it to `window.map`
map_init_js = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    for (var key in window) {
        if (key.startsWith("map_")) {
            window.map = window[key];
            break;
        }
    }
});
</script>
"""

# HTML control panel
panel_html = """
<div id="custom-controls" style="
    position: absolute;
    top: 10px;
    left: 10px;
    z-index: 9999;
    background: white;
    padding: 10px;
    border-radius: 8px;
    box-shadow: 0 0 5px rgba(0,0,0,0.3);
    font-family: sans-serif;
">
    <h4>The Board:</h4>



    
    #<button onclick="window.map.setView([40.7128, -74.0060], 12)">New York</button><br><br>
    #<button onclick="window.map.setView([34.0522, -118.2437], 12)">Los Angeles</button><br><br>
    #<button onclick="window.map.setView([51.5074, -0.1278], 12)">London</button>
</div>
"""

# Inject both
m.get_root().html.add_child(folium.Element(map_init_js))
m.get_root().html.add_child(folium.Element(panel_html))




# Save it
m.save("map_with_controls2.html")


In [ ]:
"""<table border="1" cellpadding="8" cellspacing="0">
  <thead>
    <tr>
      <th>Name</th>
      <th>Height</th>
      <th>Bags</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><button onclick="window.map.setView([56.796891, -5.003675], 12)">Ben Nevis [Beinn Nibheis]</button></td>
      <td>1344m</td>
     <td><img src="Scott.png" alt="Description" width="100"></td>
    </tr>
    <tr>
      <td><button onclick="window.map.setView([58.413113, -4.60786], 12)">Ben Hope</button></td>
      <td>927m</td>
     <td></td>
    </tr>
    <tr>
      <td><button onclick="window.map.setView([56.666942, -4.10023], 12)">Schiehallion</button></td>
      <td>1083</td>
     <td><img src="Scott.png" alt="Description" width="100"><img src="Matthew.png" alt="Description" width="100"></td>
    </tr>
  </tbody>
</table>"""

In [ ]:
<td><button onclick="window.map.setView([58.413113, -4.60786], 12)">Ben Hope</button></td>
<td><button onclick="window.map.setView([56.666942, -4.10023], 12)">Schiehallion</button></td>

In [ ]:
matthew

In [4]:
scotland_center = [56.4907, -4.2026]

# Create the map centered on Scotland
m = folium.Map(location=scotland_center, zoom_start=6)

In [5]:
bag_icon = folium.CustomIcon(icon_image="bag_icon.png",
                            icon_size=(16, 16),
                            icon_anchor=(8, 8)
                            )

no_bag_icon = folium.CustomIcon(icon_image="gap_icon.png",
                            icon_size=(14, 14),
                            icon_anchor=(7, 7)
                            )

In [6]:
def add_marker(row):

    name = row["Name"]
    height = row["Metres"]
    coords = [row["Latitude"], row["Longitude"]]
    year = row["Matthew"]

    if year != "":
        icon = bag_icon
        label = "<br>".join([name, f"height: {height}m", f"bagged: {year}"])
    else:
        icon = no_bag_icon
        label = "<br>".join([name, f"height: {height}m"])
        
    folium.Marker(
    location = coords,
    tooltip = label,
    icon = icon
    ).add_to(m)

    return

In [7]:
for _, row in df.head(10).iterrows():
    add_marker(row)

In [8]:
m.get_root().html.add_child(folium.Element("""
<script>
    // Expose the map after it's created
    window.addEventListener('load', () => {
        window.myMap = map;

        // Listen for messages from parent
        window.addEventListener('message', (event) => {
            try {
                const data = JSON.parse(event.data);
                if (data.type === 'goto') {
                    map.setView([data.lat, data.lng], data.zoom);
                }
            } catch (e) {
                console.error('Invalid message:', e);
            }
        });
    });
</script>
"""))

m.save("map.html")

In [9]:
m.save("index.html")

In [11]:
df

,Name,Region,County,Metres,Feet,Latitude,Longitude,Matthew
0,Ben Nevis [Beinn Nibheis],04A: Fort William to Loch Treig & Loch Leven,Highland,1344,4411,56.796891,-5.003675,
1,Ben Macdui [Beinn Macduibh],08A: Cairngorms,Aberdeenshire/Moray,1309,4296,57.070368,-3.669099,2024
2,Braeriach,08A: Cairngorms,Aberdeenshire/Highland,1296,4252,57.078298,-3.728373,
3,Cairn Toul,08A: Cairngorms,Aberdeenshire,1291,4236,57.054406,-3.710757,
4,Sgor an Lochain Uaine,08A: Cairngorms,Aberdeenshire,1258,4127,57.058376,-3.725897,
...,...,...,...,...,...,...,...,...
277,Meall na Teanga,10C: Loch Arkaig to Glen Moriston,Highland,916,3008,56.98903,-4.93094,
278,Beinn a' Chleibh,01D: Inveraray to Crianlarich,Argyll and Bute/Stirling,916,3006,56.39023,-4.83563,
279,Ben Vane,01D: Inveraray to Crianlarich,Argyll and Bute,915,3004,56.249786,-4.781655,
280,Carn Aosda,06B: Pitlochry to Braemar & Blairgowrie,Aberdeenshire,915,3003,56.895696,-3.423274,2024


In [ ]:
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8" />
  <title>Folium Map in Panel</title>
  <style>
    html, body {
      margin: 0;
      height: 100%;
      font-family: sans-serif;
    }

    .container {
      display: flex;
      height: 100%;
    }

    .panel {
      width: 300px;
      background: #f8f8f8;
      padding: 1em;
      box-shadow: 2px 0 5px rgba(0,0,0,0.1);
      overflow-y: auto;
    }

    .map-frame {
      flex: 1;
      border: none;
    }
  </style>
</head>
<body>
  <div class="container">
    <div class="panel">
      <h2>Info Panel</h2>
      <p>This panel can contain filters, stats, or controls.</p>
    </div>
    <iframe src="index.html" class="map-frame"></iframe>
  </div>
</body>
</html>


In [13]:
import folium

# Create map
m = folium.Map(location=[40, -100], zoom_start=4)

# JavaScript to extract the internal map object and assign it to `window.map`
map_init_js = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    for (var key in window) {
        if (key.startsWith("map_")) {
            window.map = window[key];
            break;
        }
    }
});
</script>
"""

# HTML control panel
panel_html = """
<div id="custom-controls" style="
    position: absolute;
    top: 10px;
    left: 10px;
    z-index: 9999;
    background: white;
    padding: 10px;
    border-radius: 8px;
    box-shadow: 0 0 5px rgba(0,0,0,0.3);
    font-family: sans-serif;
">
    <h4>Go To:</h4>
    <button onclick="window.map.setView([40.7128, -74.0060], 12)">New York</button><br><br>
    <button onclick="window.map.setView([34.0522, -118.2437], 12)">Los Angeles</button><br><br>
    <button onclick="window.map.setView([51.5074, -0.1278], 12)">London</button>
</div>
"""

# Inject both
m.get_root().html.add_child(folium.Element(map_init_js))
m.get_root().html.add_child(folium.Element(panel_html))

# Save it
m.save("map_with_controls2.html")
